In [2]:
""" Strip all metadata, enforce 1600px limit on dimensions, compress by 75%
"""

' Strip all metadata, enforce 1600px limit on dimensions, compress by 75%\n'

In [3]:
from PIL import Image, ExifTags, TiffImagePlugin
# import json
import os
from OsOps import Ops

In [4]:
# if image above 1600 x 1600, reduce to comply
def limited_dims( orig_size, limit = 1600 ):
    mDim = max( orig_size ) # get largest dimension
    # if exceed limit, reduce both dims by same divisor
    # any float dim to int will distort up to a single pixel of width or height
    if mDim > limit: return tuple( int( d / ( mDim / limit ) ) for d in orig_size )
    else: return orig_size

In [5]:
# Img = Image.open(f'ORDUP\\4 IMG_4357 copy.jpg')

def getDT_taken( img ):
    dt = img._getexif()[36867]  # code returns "taken" date
    dt = "_".join( dt.split(' ') )
    return "".join( dt.split(':') ) 

# print ( [ s for s in Img.getexif().keys() ] )
# print ( [ s for s in Img.getexif().values() ] )

In [6]:
def get_files( direc ):
    return [ fname for fname in (file for file in os.listdir( direc )
        if os.path.isfile(os.path.join( direc, file))) ]

In [7]:
originals_dir = 'ORDUP'

for FName in ( fns := get_files( originals_dir ) ):
    print( f"output expected to take {len( fns ) * 78 } seconds" )
    Img = Image.open(f'{originals_dir}\\{FName}')
    img_new = Image.new(Img.mode, Img.size)  # get new image model
    img_new.putdata(Img.getdata())  # supply image data only, no exif
    if 'P' in Img.mode: img_new.putpalette(Img.getpalette()) # Copy any palette
    img_new_rsz = img_new.resize( limited_dims( Img.size ) )
    img_new_rsz.save(f'STRIPPED\\RSTR_{getDT_taken( Img )}_PROC_{Ops().dtStamp()}.jpg')


# print expectation on func start
# 2 mins 19.6 secs for 180 items
# 2 mins 20.4 secs for 180 items


output expected to take 78 seconds


In [48]:
# def store_exif( fname, img_obj, exif_dict ):
#     exif_dict[ fname ] = {
#         ExifTags.TAGS[k]: v
#         for k, v in img_obj._getexif().items()
#         if k in ExifTags.TAGS 
#         }

# store_exif( FName, Img, Exif_dict )  # keep exif for pers reference

# # cast unserializable types inj exif dict
# Exif_dict_cstd = {}

# for k, v in Exif_dict.items():
#     Exif_dict_cstd[ k ] = { kk : vv
#             if kk in ExifTags.TAGS and type(v) is not bytes
#             if type(vv) not in [bytes, TiffImagePlugin.IFDRational] 
#             else str(vv)
#             for kk, vv in v.items()
#             }

# with open('Exif_dict.json', 'w') as fp: json.dump(Exif_dict_cstd, fp)
